# About

This notebook adds canopy height features to the points sampled in the notebook `2_sample_pts_from_polygos.ipynb`. The notebook assumes the csv files with the sampled points  are located in the 'temp' folder. The canopy height rasters for Santa Barbara County were obtained from the California Forest Observatory (CFO) using the `1_download_CFO_canopy_height_raster.ipynb` notebook and are located in the 'SantabarbaraCounty_lidar' folder. 


In the process of adding the canopy height features, this notebook creates four additional temporary rasters from the CFO canopy height layer *H*. These layers are avg_lidar, max_lidar, min_lidar, and min_max_diff. For a given year, the avg_lidar layer is created by replacing the value of a pixel *p* in *H* by the average of the values of *H* in a 3x3 window centered at *p* (effectively a convolution of the raster *H* with a 3x3 matrix with constant weights 1/9). The max_lidar is created by replacing the value of a pixel *p* in *H* with the maximum value of *H* in a 3x3 window centered at *p*. The min_lidar layer is created similarly, now taking the minimum value over the window. Finally, the min_max_diff layer is the difference between the max_lidar and the min_lidar layers. All the functions to create these raster layers and sample information from them are in `lidar_sampling_functions`. 

To add canopy height features to all points from all aois and all years, you will need to run the notebook once per year, using all the aois.

**NOTEBOOK VARIABLES:**

- `aoi_year` (int): the year of the points which will have lidar features added. Must be one of 2012, 2014, 2016, 2018 or 2020.

- `aois` (array): the areas of interest of the points which will have lidar features added. Must be a subset of `['campus_lagoon','carpinteria','gaviota', 'point_conception']`.

- `lidar_year` (int): the year of canopy height data from which to sample the lidar features for the points. Must be 2016, 2018, or 2020. Ideally, `aoi_year = lidar_year`, but due to data availability it is recommended to make `lidar_year=2016` when `aoi_year` equals 2014 or 2012. 

- `delete_pts` (bool): whether to delete the files with the original points or not.

Notes: there are no points sampled from point_conception on 2016. The notebook automatically excludes this option. 


**OUTPUT:**
For each csv of points from the specified year and aoi, the notebook creates a dataframe with the original features from the intial points dataset (see notebook `2_sample_pts_from_polygons`) augmented with the canopy height, avg_lidar, max_lidar, min_lidar, and min_max_diff features sampled at the points location from the CFO canopy height raster from year `lidar_year`. 
The dataframes are then saved each as a csv file in the 'temp' folder.

In [1]:
import os
import pandas as pd
import geopandas as gpd
import rioxarray as riox

import rasterio

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

# Specify notebook variables

In [18]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

aoi_year = 2012

aois = ['campus_lagoon','carpinteria','gaviota','point_conception']

lidar_year = 2016 # must be 2016, 2018 or 2020

delete_pts = True
    
# ***************************************************
# ***************************************************

In [19]:
# If aoi_year=2016, then use aois without point_conception (there are no 2016 polygons in that aoi)
if ('point_conception' in aois) and (aoi_year == 2016):
    aois.remove('point_conception')

# Open canopy height raster and create auxiliary min, max, and avg rasters

In [20]:
lidar_rast_r = rasterio.open(lsf.path_to_lidar(lidar_year))

lsf.save_min_max_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'temp'),
                              year = lidar_year)

lsf.save_avg_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'temp'),
                              year = lidar_year)


# Add canopy height data from year `lidar_year` to points from all aois in year `aoi_year`

In [21]:
for aoi in aois:
    pts_fp = os.path.join(os.getcwd(), 
                          'temp',
                          aoi+'_points_'+str(aoi_year)+'.csv')

    ## Obtain CRS from itemid and create geodataframe from points
    itemid = pd.read_csv(pts_fp).naip_id[0]
    pts = lsf.geodataframe_from_csv(pts_fp, lsf.crs_from_itemid(itemid))

    ## Sample LIDAR
    pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)
    lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)
    
    # file paths to auxiliary LIDAR rasters
    lidar_fps = []
    for tag in ['maxs_', 'mins_', 'avgs_']:
        lidar_fps.append(os.path.join(os.getcwd(),
                                     'temp',
                                     'lidar_'+tag+ str(lidar_year)+'.tif'))

    ## Sample mmax, min and avg LIDAR
    maxs_rast_r = rasterio.open(lidar_fps[0])
    max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

    mins_rast_r = rasterio.open(lidar_fps[1])
    min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

    avg_rast_r = rasterio.open(lidar_fps[2])
    avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)
    
    ## Add all LIDAR data to pts dataframe
    pts['lidar'] = lidar_samples
    pts['max_lidar']= max_samples
    pts['min_lidar'] = min_samples
    pts['min_max_diff'] = pts.max_lidar - pts.min_lidar  # include difference
    pts['avg_lidar'] = avg_samples

    ## Save points with added LIDAR data
    ptslidar_fp = os.path.join(os.getcwd(), 
                               'temp', 
                               aoi +'_pts_spectral_lidar_'+str(aoi_year)+'.csv')
    pts.to_csv(ptslidar_fp)
    
    ## # Delete point csv files (ones without LIDAR)
    if delete_pts == True:
        os.remove(pts_fp)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interfac

# Delete auxiliary LIDAR rasters created in the beginning 

In [ ]:
for fp in lidar_fps:
    os.remove(fp)